# 사이킷런 모델 생성

In [1]:
!pip install scikit-learn

In [2]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [30]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [3]:
# iris datasets 로딩
iris = load_iris()

iris_data  = iris.data # feature
iris_label = iris.target # label

iris_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
iris_pdf = pd.DataFrame(iris_data, columns=iris_columns)
iris_pdf['target'] = iris_label
iris_pdf

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [4]:
iris_pdf.to_csv('./data/iris.csv', index=False)

In [5]:
# 데이터 분할 및 모델 생성
# from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier # Estimator
from sklearn.model_selection import train_test_split # RandomSpliter

X_train, X_test, t_train, t_test = train_test_split(
    iris_data,
    iris_label,
    test_size=0.2,
    random_state=42
)

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, t_train) # 훈련! tree_clf 모델 자체에서 훈련이 일어나게 된다.

pred = tree_clf.predict(X_test)
print(pred)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


# Spark ML 사용하기
- 100% dataframe 기반
- scikit learn은 numpy, pandas 기반

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('tree-clf').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/13 04:30:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/13 04:30:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/13 04:30:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/13 04:30:25 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/06/13 04:30:25 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [7]:
iris_filepath = "/home/ubuntu/working/spark-examples/data/iris.csv"
iris_sdf = spark.read.csv(f'file://{iris_filepath}',
                         inferSchema = True,
                         header = True)
iris_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         5.1|        3.5|         1.4|        0.2|     0|
|         4.9|        3.0|         1.4|        0.2|     0|
|         4.7|        3.2|         1.3|        0.2|     0|
|         4.6|        3.1|         1.5|        0.2|     0|
|         5.0|        3.6|         1.4|        0.2|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



In [8]:
# randomSplit 메소드를 활용해 훈련/ 테스트 데이터 세트 분할

train_sdf, test_sdf = iris_sdf.randomSplit([0.8,0.2], seed=42)

## cache()

In [9]:
# 훈련 데이터 세트는 어떻게 변환이 되어도 하나만 존재하는게 좋다.
# -> 모델을 여러개 사용해서 변환이 되는 상황
# 여러개의 훈련 데이터 프레임이 생성되는 것을 방지해야 한다

# 훈련 데이터가 모델에 들어가게 되면 transform이 일어나게 된다.
# 여러번의 훈련을 거치게 되면, transform이 여러 번 일어나게 된다.
# ---> train_sdf가 메모리 내에 여러개 똑같은 것이 생길 수 있다.

# 따.라.서!!! 훈련 직전에 사용할 데이터는 캐싱을 하는게 좋다.
train_sdf.cache()
train_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         4.3|        3.0|         1.1|        0.1|     0|
|         4.4|        2.9|         1.4|        0.2|     0|
|         4.4|        3.2|         1.3|        0.2|     0|
|         4.5|        2.3|         1.3|        0.3|     0|
|         4.6|        3.1|         1.5|        0.2|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



## VectorAssembler

- 이 때 `벡터 어셈블러` 를 이용해서 한 row에 있는 데이터를 묶어줄 것임.
- 현재에는 sepal_length, sepal_width..등 데이터가 모두 떨어져 있는 상태

`VectorAssembler` 를 이용해 모든 feature 컬럼을 하나의 feature vector로 만든다.(행벡터)

In [10]:
from pyspark.ml.feature import VectorAssembler

# VectorAssembler 로 데이터프레임에 있는 데이터를 하나의 행벡터로 합쳐준다.


# inputCols = 합칠 컬럼의 목록들
# outputCol = 데이터가 합쳐진 컬럼의 이름. assemble이 완료된 컬럼

iris_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

# inputCols에 집어넣은 컬럼들이 outputCol인 features라는 이름으로 벡터로 묶일 것
vec_assembler = VectorAssembler(inputCols=iris_columns, outputCol = 'features')

# VectorAssembler Transform
train_feature_vector_sdf = vec_assembler.transform(train_sdf)
train_feature_vector_sdf.show(5)

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.3|        3.0|         1.1|        0.1|     0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|     0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.2|         1.3|        0.2|     0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|     0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|     0|[4.6,3.1,1.5,0.2]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



# Estimator

spark ML의 모델은 추정기(Estimator)지만, 데이터를 변환시키는 Transformer에 해당한다.
- train데이터를 받아서 예측값(prediction)으로 변환을 시키는 transform 과정이 일어나기 때문

### Decision Tree

In [35]:
from pyspark.ml.classification import DecisionTreeClassifier

# 모델 생성: "데이터 프레임의 어떤 컬럼"의 데이터를 이용해서 학습을 할지 결정 지어줘야 한다.
dt = DecisionTreeClassifier(
    featuresCol = 'features',
    labelCol = 'target',
    maxDepth = 2,
    
)
type(dt)

pyspark.ml.classification.DecisionTreeClassifier

In [36]:
# 모델 학슴. fit() 메소드를 이용하여 학습을 수행하고, 그 결과를 ML 모델로 변환한다.
dt_model = dt.fit(train_feature_vector_sdf)
type(dt_model)

pyspark.ml.classification.DecisionTreeClassificationModel

In [37]:
# 테스트 데이터 예측
test_sdf.show(5)

# 훈련 데이터에서 적용시켰던 Transformer를 테스트 세트에다가도 그대로 적용시킨다.
test_feature_vector_sdf = vec_assembler.transform(test_sdf)
test_feature_vector_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         4.4|        3.0|         1.3|        0.2|     0|
|         4.6|        3.2|         1.4|        0.2|     0|
|         4.6|        3.6|         1.0|        0.2|     0|
|         4.8|        3.1|         1.6|        0.2|     0|
|         4.9|        3.1|         1.5|        0.1|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|
|         4.6|        3.6|         1.0|        0.2|     0|[

In [38]:
# 예측
predictions = dt_model.transform(test_feature_vector_sdf)
predictions.show()

+------------+-----------+------------+-----------+------+-----------------+--------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features| rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------+--------------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|       

- rawPrediction: 각 붓꽃 종류에 대한 예측 점수
- probability: 각 붓꽃 종류에 대한 예측 확률
- prediction: 예측된 붓꽃 종류의 레이블
    - 즉, iris.target_names를 보면 (['setosa', 'versicolor', 'virginica'],  꽃 종류는 이렇게 세가지
    - 

- `rawPrediction`: 머신러닝 모델 알고리즘 별로 다를 수 있다.
   - 머시널닝 알고리즘에 의해 계산된 값
   - 값에 대한 정확한 의미는 없음.
   - `LogisticRegression` 의 경우 예측  label 별로, 예측 수행 전 `sigmoid` 함수 적용 전 값
       - y hat = 

# 모델 평가

In [40]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol='target',
    predictionCol='prediction',
    metricName = 'accuracy'
)

accuracy = evaluator_accuracy.evaluate(predictions)
accuracy # maxDepth 수 많아지면 정확도 올라가는 경향?

0.9583333333333334

### Logistic Regression

- 내 답

In [52]:
from pyspark.ml.classification import LogisticRegression

# 벡터 어셈블
# 모델 생성
lr = LogisticRegression(
        featuresCol='features',
        labelCol='target', maxIter=10)

# 모델 학습
lr_model = lr.fit(train_feature_vector_sdf)

# 테스트 데이터 변환
test_feature_vector_lr = vec_assembler.transform(test_sdf)

# 예측
lr_pred = lr_model.transform(test_feature_vector_lr)

# 모델 평가
lr_eval = MulticlassClassificationEvaluator(
        labelCol='target',
        predictionCol='prediction',
        metricName = 'accuracy'
)

accuracy_lr = lr_eval.evaluate(lr_pred)
accuracy_lr

1.0

- 강사님 답

In [54]:
# LogisticRegression 사용해 보기[실습]
from pyspark.ml.classification import LogisticRegression

# ML 알고리즘 객체 생성
lr = LogisticRegression(featuresCol='features', labelCol='target', maxIter=10)

lr_model = lr.fit(train_feature_vector_sdf)

predictions = lr_model.transform(test_feature_vector_sdf)
predictions.show(5)

accuracy = evaluator_accuracy.evaluate(predictions)
print("정확도", accuracy)

+------------+-----------+------------+-----------+------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------------+--------------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[18.6086266693526...|[0.99997762791224...|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[18.8180066107263...|[0.99997581287298...|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[22.6963845270307...|[0.99999942608846...|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[16.7506644665745...|[0.99971232954776...|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[17.3393987944

# 파이프라인 구축

In [58]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

iris_columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

# Pipeline은 개별 변환 및 모델 학습 작업을 각각의 stage로 정의해서 파이프라인에 순서대로 등록
# Pipeline.fit() 메소드를 활용해서 순서대로 연결된 스테이지 작업을 일괄적으로 수행
# Pipeline.fit()의 결과물은 PipelineModel로 반환이 된다.
# PipelineModel에서 예측 작업을 transform()으로 수행

# 첫 번째 stage는 Feature Vectorization을 위한 VectorAssembler
stage_1 = VectorAssembler(inputCols=iris_columns, outputCol = 'features')

# 두번째 stage는 학습을 위한 모델 생성
stage_2 = DecisionTreeClassifier(featuresCol='features', labelCol='target', maxDepth = 3)

# 리스트를 활용해 stage를 순서대로 배치
stages = [ stage_1, stage_2 ]

# 파이프라인에 등록
pipeline = Pipeline(stages = stages)
type(pipeline)


pyspark.ml.pipeline.Pipeline

In [59]:
pipeline_model = pipeline.fit(train_sdf)
type(pipeline_model)

pyspark.ml.pipeline.PipelineModel

In [60]:
# 파이프라인을 통해서 테스트 세트 예측 
predictions = pipeline_model.transform(test_sdf)
predictions.show(5)

+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
+------------+-----------+------------+-----------+------+------

In [61]:
spark.stop()